# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`


In [2]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [3]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [4]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'config', 'local', 'met', 'uk_food']


In [5]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [6]:
# review the collections in our new database
print("Collections in uk_food database:")
print(db.list_collection_names())

Collections in uk_food database:
['establishments']


In [7]:
# review a document in the establishments collection
establishments = db['establishments']

pprint(establishments.find_one())

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('663ac84802c3408abaeff8ef'),
 'geocode': {'latitude': 51.083812, 'longitude': 1.195625},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': None,


In [8]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [9]:
# Create a dictionary for the new restaurant data
new_restaurant_data = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "BusinessTypeID": "",
    "AddressLine1": "Penang Flavours",
    "AddressLine2": "146A Plumstead Rd",
    "AddressLine3": "London",
    "AddressLine4": "",
    "PostCode": "SE18 7DY",
    "Phone": "",
    "LocalAuthorityCode": "511",
    "LocalAuthorityName": "Greenwich",
    "LocalAuthorityWebSite": "http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress": "health@royalgreenwich.gov.uk",
    "scores": {
        "Hygiene": "",
        "Structural": "",
        "ConfidenceInManagement": ""
    },
    "SchemeType": "FHRS",
    "geocode": {
        "longitude": "0.08384000",
        "latitude": "51.49014200"
    },
    "RightToReply": "",
    "Distance": 4623.9723280747176,
    "NewRatingPending": True
}


In [10]:
# Insert the new restaurant into the collection
result = establishments.insert_one(new_restaurant_data)

In [11]:
# Check that the new restaurant was inserted
if result.inserted_id:
    print("New restaurant 'Penang Flavours' inserted into the collection.")
else:
    print("Failed to insert the new restaurant.")

New restaurant 'Penang Flavours' inserted into the collection.


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [12]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType": "Restaurant/Cafe/Canteen"}

projection = {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}

result = establishments.find_one(query, projection)

print(result)

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [13]:
# Update the new restaurant with the correct BusinessTypeID
query = {"BusinessType": "Restaurant/Cafe/Canteen"}
projection = {"BusinessTypeID": 1, "_id": 0}
business_type_info = establishments.find_one(query, projection)

business_type_id = business_type_info["BusinessTypeID"]

# Update the new restaurant document with the correct BusinessTypeID
update_query = {"BusinessName": "Penang Flavours"}
update_data = {"$set": {"BusinessTypeID": business_type_id}}
update_result = establishments.update_one(update_query, update_data)

# Check if the update was successful
if update_result.modified_count > 0:
    print("BusinessTypeID updated successfully for 'Penang Flavours'.")
else:
    print("Failed to update BusinessTypeID for 'Penang Flavours'.")

Failed to update BusinessTypeID for 'Penang Flavours'.


In [14]:
# Confirm that the new restaurant was updated
updated_restaurant = establishments.find_one({"BusinessName": "Penang Flavours"})
pprint(updated_restaurant)

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('663ac86dda7a58e0c4c35eca'),
 'geocode': {'latitude': 51.490142, 'longitude': 0.08384},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [15]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})

# Print the number of documents
print("Number of documents with LocalAuthorityName as 'Dover':", dover_count)

Number of documents with LocalAuthorityName as 'Dover': 0


In [16]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = establishments.delete_many({"LocalAuthorityName": "Dover"})

# Check if documents were deleted successfully
if delete_result.deleted_count > 0:
    print("Documents with LocalAuthorityName as 'Dover' deleted successfully.")
else:
    print("No documents with LocalAuthorityName as 'Dover' found.")

# Confirm the number of remaining documents after deletion
remaining_count = establishments.count_documents({})
print("Number of remaining documents in the collection:", remaining_count)

No documents with LocalAuthorityName as 'Dover' found.
Number of remaining documents in the collection: 38787


In [17]:
# Check if any remaining documents include Dover
remaining_dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})

# Check if any remaining documents still include Dover
if remaining_dover_count == 0:
    print("No remaining documents include 'Dover' in the LocalAuthorityName field.")
else:
    print("There are still", remaining_dover_count, "remaining documents with 'Dover' in the LocalAuthorityName field.")

No remaining documents include 'Dover' in the LocalAuthorityName field.


In [18]:
# Check that other documents remain with 'find_one'
remaining_document = establishments.find_one()

# Display the remaining document
pprint(remaining_document)

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('663ac84802c3408abaeff8ef'),
 'geocode': {'latitude': 51.083812, 'longitude': 1.195625},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': None,


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [19]:
# Change the data type from String to Decimal for longitude and latitude
update_pipeline = [
    {"$set": {"geocode.longitude": {"$toDouble": "$geocode.longitude"}}},
    {"$set": {"geocode.latitude": {"$toDouble": "$geocode.latitude"}}}
]

# Update documents to convert longitude and latitude to decimal numbers
update_result = establishments.update_many({}, update_pipeline)

# Check if the update was successful
if update_result.modified_count > 0:
    print("Longitude and latitude data type converted to Decimal successfully.")
else:
    print("No documents were updated.")

# Print the number of documents updated
print("Number of documents updated:", update_result.modified_count)

Longitude and latitude data type converted to Decimal successfully.
Number of documents updated: 1


Use `update_many` to convert `RatingValue` to integer numbers.

In [20]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [21]:
# Change the data type from String to Integer for RatingValue
update_result = establishments.update_many({"RatingValue": {"$exists": True}}, [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}])

# Check if the update was successful
if update_result.modified_count > 0:
    print("RatingValue data type converted to Integer successfully.")
else:
    print("No documents were updated.")

# Print the number of documents updated
print("Number of documents updated:", update_result.modified_count)

No documents were updated.
Number of documents updated: 0


In [22]:
# Check that the coordinates and rating value are now numbers
cursor = establishments.find({}, limit=5)

# Iterate over the cursor and print the relevant fields
for document in cursor:
    print("BusinessName:", document["BusinessName"])
    print("Longitude:", document["geocode"]["longitude"])
    print("Latitude:", document["geocode"]["latitude"])
    print("RatingValue:", document["RatingValue"])
    print()

BusinessName: The Pavilion
Longitude: 1.195625
Latitude: 51.083812
RatingValue: 5

BusinessName: Wear Bay Bowls Club
Longitude: 1.196408
Latitude: 51.086058
RatingValue: 4

BusinessName: St Marys COE (aided) Primary School
Longitude: 1.194762
Latitude: 51.085797
RatingValue: 5

BusinessName: The Ship
Longitude: 1.188537
Latitude: 51.08084
RatingValue: 5

BusinessName: Mariner
Longitude: 1.188537
Latitude: 51.08084
RatingValue: 5

